In [1]:
### https://github.com/xianyi/OpenBLAS/issues/3237
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from cmfrec import CMF

df_full = pd.read_csv("ratings.dat", delimiter="::", engine="python", header=None)
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_full.columns = ["UserId", "ItemId", "Rating", "Timestamp"]
df_full = df_full.drop("Timestamp", axis=1)
df_full["UserId"], _ = pd.factorize(df_full["UserId"])
df_full["ItemId"], _ = pd.factorize(df_full["ItemId"])
df_train["UserId"], users_train = pd.factorize(df_train["UserId"])
df_train["ItemId"], items_train = pd.factorize(df_train["ItemId"])
df_test["UserId"] = pd.Categorical(df_test["UserId"], users_train).codes
df_test["ItemId"] = pd.Categorical(df_test["ItemId"], items_train).codes

X_full = coo_matrix((df_full.Rating, (df_full.UserId, df_full.ItemId)))
X_train = coo_matrix((df_train.Rating, (df_train.UserId, df_train.ItemId)))

def convert_sp_dtypes(X):
    X.row = X.row.astype(np.int32)
    X.col = X.col.astype(np.int32)
    X.data = X.data.astype(np.float64)
    return X
X_full = convert_sp_dtypes(X_full)
X_train = convert_sp_dtypes(X_train)

In [3]:
%%timeit -r 1 -n 1 -p 7 -o
### Variation 1: Conjugate Gradient method, with biases
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=True, finalize_chol=False,
            verbose=False, precompute_for_predictions=False)
model.fit(X_full)

13.70771 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 13.70771 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [4]:
%%timeit -r 1 -n 1 -p 7 -o
### Variation 2: Conjugate Gradient method, no biases
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=True, finalize_chol=False,
            user_bias=False, item_bias=False,
            verbose=False, precompute_for_predictions=False)
model.fit(X_full)

11.83269 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 11.83269 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [5]:
%%timeit -r 1 -n 1 -p 7 -o
### Variation 3: Cholesky method, with biases
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=False,
            verbose=False, precompute_for_predictions=False)
model.fit(X_full)

31.23867 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 31.23867 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [6]:
%%timeit -r 1 -n 1 -p 7 -o
### Variation 4: CG method, with biases, and implicit features
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=True, finalize_chol=False,
            add_implicit_features=True,
            verbose=False, precompute_for_predictions=False)
model.fit(X_full)

23.0416 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 23.0416 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [7]:
%%timeit -r 1 -n 1 -p 7 -o
### Variation 5: Cholesky method, with biases, and implicit features
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=False,
            add_implicit_features=True,
            verbose=False, precompute_for_predictions=False)
model.fit(X_full)

38.13179 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 38.13179 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [8]:
def print_rmse(pred, real):
    errors = pred - real
    rmse = np.sqrt(np.mean(np.array(errors) ** 2))
    print("RMSE: %f" % rmse)

In [9]:
### Variation 1: Conjugate Gradient method, with biases
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=True, finalize_chol=False,
            verbose=False, precompute_for_predictions=False)
model.fit(X_train)
pred = model.predict(df_test.UserId, df_test.ItemId)
print_rmse(pred, df_test.Rating)

RMSE: 0.788233


In [10]:
### Variation 2: Conjugate Gradient method, no biases
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=True, finalize_chol=False,
            user_bias=False, item_bias=False,
            verbose=False, precompute_for_predictions=False)
model.fit(X_train)
pred = model.predict(df_test.UserId, df_test.ItemId)
print_rmse(pred, df_test.Rating)

RMSE: 0.791481


In [11]:
### Variation 3: Cholesky method, with biases
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=False,
            verbose=False, precompute_for_predictions=False)
model.fit(X_train)
pred = model.predict(df_test.UserId, df_test.ItemId)
print_rmse(pred, df_test.Rating)

RMSE: 0.786923


In [12]:
### Variation 4: CG method, with biases, and implicit features
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=True, finalize_chol=False,
            add_implicit_features=True,
            verbose=False, precompute_for_predictions=False)
model.fit(X_train)
pred = model.predict(df_test.UserId, df_test.ItemId)
print_rmse(pred, df_test.Rating)

RMSE: 0.785427


In [13]:
### Variation 5: Cholesky method, with biases, and implicit features
model = CMF(k=50, lambda_=0.05, scale_lam=True, niter=15,
            use_cg=False,
            add_implicit_features=True,
            verbose=False, precompute_for_predictions=False)
model.fit(X_train)
pred = model.predict(df_test.UserId, df_test.ItemId)
print_rmse(pred, df_test.Rating)

RMSE: 0.782414
